## Task 3 - Sentiment Analysis
Use google BERT word embedding model to build a sentiment analysis model.


#### Datasets collected and used
* TEAD sample data
* LABR dataset
* ArSAS dataset
* AJGT dataset
* 40000 Egyptian tweet dataset
* SemEval2017-task4 subtask-A arabic dataset
* ASTD dataset

#### Labels
* __0__: Negative Sentiment
* __1__: Neutral Sentiment
* __2__: Positive Sentiment


####  Combined dataset
* 693,917 total records
    * __Positive__: 462,133
    * __Neutral__: 29059
    * __Negative__: 202,725

In [1]:
import pandas as pd

### 1- Preparing the Arabic sentiment analysis dataset

#### 1.1 TEAD sample data

##### 1.1.1 Loading positive sentences

In [2]:
tead_pos_df = pd.read_csv("data/sentiment-analysis/TEAD_sample_data_final_positive_clean.txt", header=None, names=["sentence"])

In [3]:
tead_pos_df

,sentence
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...
...,...
392482,عاد مارسييلو إلي نعرفه دفاع هجوم إبداع إمتاع إ...
392483,قلت أحبك وقلبي قال بين المحاني والله إن كلمة أ...
392484,انا وحنان بنتكلم واتس وماسنجر و ف نفس الوقت وف...
392485,الآ بذكر الله تطمئن القلؤب غرد الان بذكر الله


In [4]:
tead_pos_df["label"] = 2

In [5]:
tead_pos_df

,sentence,label
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...,2
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...,2
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم,2
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...,2
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...,2
...,...,...
392482,عاد مارسييلو إلي نعرفه دفاع هجوم إبداع إمتاع إ...,2
392483,قلت أحبك وقلبي قال بين المحاني والله إن كلمة أ...,2
392484,انا وحنان بنتكلم واتس وماسنجر و ف نفس الوقت وف...,2
392485,الآ بذكر الله تطمئن القلؤب غرد الان بذكر الله,2


##### 1.1.2 Loading negative sentences

In [6]:
tead_neg_df = pd.read_csv("data/sentiment-analysis/TEAD_sample_data_final_negative_clean.txt", header=None, names=["sentence"])

In [7]:
tead_neg_df

,sentence
0,السعوديين شافو ترمب و مومصدقين نفسهم لعمى تقلو...
1,شكرا ويستهام لك كل الاحترام
2,تضعك المعرفة في صفوف الحكماء و يضعك العمل في ص...
3,خالي الف رحمةة ع روحةة
4,بيريز جيب بونوتشي يالذيب مانبي هجوم
...,...
163431,وش حاجتكك بقلبي يلي خذيته كلي ولهه
163432,لكل شيء لابد نهاية و هذي النهاية ي تشابي
163433,كنت عشمانة في ليله حلوة بس الظروف ليها رأي اخر
163434,البندر قبل قال بيعرض صوت ام حمود وهي متوفيه اصلن


In [8]:
tead_neg_df["label"] = 0

In [9]:
tead_neg_df

,sentence,label
0,السعوديين شافو ترمب و مومصدقين نفسهم لعمى تقلو...,0
1,شكرا ويستهام لك كل الاحترام,0
2,تضعك المعرفة في صفوف الحكماء و يضعك العمل في ص...,0
3,خالي الف رحمةة ع روحةة,0
4,بيريز جيب بونوتشي يالذيب مانبي هجوم,0
...,...,...
163431,وش حاجتكك بقلبي يلي خذيته كلي ولهه,0
163432,لكل شيء لابد نهاية و هذي النهاية ي تشابي,0
163433,كنت عشمانة في ليله حلوة بس الظروف ليها رأي اخر,0
163434,البندر قبل قال بيعرض صوت ام حمود وهي متوفيه اصلن,0


#### 1.2 LABR dataset

* 1 and 2 star ratings will be considered as "Negative"
* 3 star ratings will be considered as "Neutral"
* 4 and 5 star ratings will be considered as "Positive"

In [10]:
labr_df = pd.read_csv("data/sentiment-analysis/LABR.tsv", sep="\t", header=None, names=["label", "id1", "id2", "id3", "sentence"])

In [11]:
labr_df

,label,id1,id2,id3,sentence
0,4,338670838,7878381,13431841,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني..."
1,4,39428407,1775679,3554772,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...
2,4,32159373,1304410,3554772,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
3,1,442326656,11333112,3554772,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
4,5,46492258,580165,3554772,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
...,...,...,...,...,...
63252,5,108789247,3926131,1420,اجمل مسرحية ألفت في تاريخ الادب الانجليزي
63253,3,513749112,16666895,7286365,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...
63254,3,137280339,4303773,7286365,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...
63255,3,175939769,4770537,7286365,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...


In [12]:
labr_df.drop(["id1", "id2", "id3"], axis=1, inplace=True)

In [13]:
labr_df

,label,sentence
0,4,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني..."
1,4,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...
2,4,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
3,1,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
4,5,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
...,...,...
63252,5,اجمل مسرحية ألفت في تاريخ الادب الانجليزي
63253,3,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...
63254,3,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...
63255,3,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...


In [14]:
labr_df = labr_df.reindex(['sentence','label'], axis=1)

In [15]:
labr_df

,sentence,label
0,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني...",4
1,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...,4
2,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...,4
3,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...,1
4,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...,5
...,...,...
63252,اجمل مسرحية ألفت في تاريخ الادب الانجليزي,5
63253,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...,3
63254,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...,3
63255,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...,3


In [16]:
labr_df['label'].loc[(labr_df['label'] < 3)] = 0

/home/tamer/anaconda3/envs/psut-nlp-course/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [17]:
labr_df['label'].loc[(labr_df['label'] == 3)] = 1

In [18]:
labr_df['label'].loc[(labr_df['label'] > 3)] = 2

In [19]:
labr_df

,sentence,label
0,"""عزازيل الذي صنعناه ،الكامن في أنفسنا"" يذكرني...",2
1,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...,2
2,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...,2
3,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...,0
4,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...,2
...,...,...
63252,اجمل مسرحية ألفت في تاريخ الادب الانجليزي,2
63253,بصراحة، لم تكن هذه الرواية على قدر توقعاتي. ا...,1
63254,هي الرواية الأولى التي قرأتها لأيميلي نصر الل...,1
63255,تَدْخل بيوت الناس، تدخل قلوبهم، تمدّ يدك تصاف...,1


In [20]:
labr_df["label"].value_counts()

2    42832
1    12201
0     8224
Name: label, dtype: int64

#### 1.3 ArSAS dataset

Assuming "Mixed" label as "Neutral".

In [21]:
arsas_df = pd.read_csv("data/sentiment-analysis/ArSAS.txt", sep="\t")

In [22]:
arsas_df

,#Tweet_ID,Tweet_text,Topic,Sentiment_label,Sentiment_label_confidence,Speech_act_label,Speech_act_label_confidence
0,929241870508724224,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Event,Positive,0.38,Assertion,0.62
1,928942264583376897,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Entity,Negative,1.00,Expression,0.68
2,928615163250520065,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Event,Positive,0.69,Assertion,1.00
3,931614713368186880,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,Event,Negative,1.00,Expression,1.00
4,929755693011427331,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,Event,Neutral,1.00,Assertion,1.00
...,...,...,...,...,...,...,...
19892,929109605267000960,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,Event,Neutral,1.00,Assertion,1.00
19893,930131859039837952,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,Event,Positive,0.69,Expression,0.69
19894,928739551136361984,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,Event,Neutral,1.00,Assertion,1.00
19895,929254320754823168,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,Event,Neutral,1.00,Assertion,1.00


In [23]:
arsas_df.drop(["#Tweet_ID", "Topic", "Sentiment_label_confidence", "Speech_act_label", "Speech_act_label_confidence"], axis=1, inplace=True)

In [24]:
arsas_df

,Tweet_text,Sentiment_label
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Positive
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Negative
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Positive
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,Negative
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,Neutral
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,Neutral
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,Positive
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,Neutral
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,Neutral


In [25]:
arsas_df.rename(columns={'Tweet_text': 'sentence', 'Sentiment_label': 'label'}, inplace=True)

In [26]:
arsas_df

,sentence,label
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Positive
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Negative
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Positive
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,Negative
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,Neutral
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,Neutral
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,Positive
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,Neutral
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,Neutral


In [27]:
arsas_df['label'].loc[(arsas_df['label'] == "Negative")] = 0

In [28]:
arsas_df['label'].loc[(arsas_df['label'] == "Neutral")] = 1

In [29]:
arsas_df['label'].loc[(arsas_df['label'] == "Mixed")] = 1

In [30]:
arsas_df['label'].loc[(arsas_df['label'] == "Positive")] = 2

In [31]:
arsas_df

,sentence,label
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,2
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,0
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,2
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,0
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,1
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,1
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,2
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,1
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,1


In [32]:
arsas_df["label"].value_counts()

1    8113
0    7384
2    4400
Name: label, dtype: int64

#### 1.4 AJGT dataset

In [33]:
ajgt_df = pd.read_excel("data/sentiment-analysis/AJGT.xlsx")

In [34]:
ajgt_df

,ID,Feed,Sentiment
0,1,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,Positive
1,2,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,Negative
2,3,كله رائع بجد ربنا يكرمك,Positive
3,4,لسانك قذر يا قمامه,Negative
4,5,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,Negative
...,...,...,...
1795,1796,يلعن شرف خواتك,Negative
1796,1797,يلعنك نذل وكلب,Negative
1797,1798,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,Positive
1798,1799,ينور عليك ويكثر من امثالك,Positive


In [35]:
ajgt_df.drop(["ID"], axis=1, inplace=True)

In [36]:
ajgt_df.rename(columns={"Feed": "sentence", "Sentiment": "label"}, inplace=True)

In [37]:
ajgt_df

,sentence,label
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,Positive
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,Negative
2,كله رائع بجد ربنا يكرمك,Positive
3,لسانك قذر يا قمامه,Negative
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,Negative
...,...,...
1795,يلعن شرف خواتك,Negative
1796,يلعنك نذل وكلب,Negative
1797,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,Positive
1798,ينور عليك ويكثر من امثالك,Positive


In [38]:
ajgt_df['label'].loc[(ajgt_df['label'] == "Negative")] = 0

In [39]:
ajgt_df['label'].loc[(ajgt_df['label'] == "Positive")] = 2

In [40]:
ajgt_df

,sentence,label
0,اربد فيها جامعات اكثر من عمان ... وفيها قد عم...,2
1,الحلو انكم بتحكوا على اساس انو الاردن ما فيه ...,0
2,كله رائع بجد ربنا يكرمك,2
3,لسانك قذر يا قمامه,0
4,​انا داشره وغير متزوجه ولدي علاقات مشبوه واحشش...,0
...,...,...
1795,يلعن شرف خواتك,0
1796,يلعنك نذل وكلب,0
1797,يمتاز الاردن بتنوع السياحه فيه فتنقسم السياحه ...,2
1798,ينور عليك ويكثر من امثالك,2


In [41]:
ajgt_df["label"].value_counts()

2    900
0    900
Name: label, dtype: int64

#### 1.5 40000 Egyptian tweets dataset

In [42]:
egypt_df = pd.read_excel("data/sentiment-analysis/40000-Egyptian-tweets.xlsx")

In [43]:
egypt_df

,review,label
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,negative
1,دائما اكره اخر ليله في كل مكان .,negative
2,يارب اللى يسرق تويتاتى يدخل النار .,negative
3,الاسراف فى تناول القهوة يسبب الوفاه .,negative
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,positive
...,...,...
39995,لنسعد ايامنا بالابتسامة بدلاً ان نملاها بالدموع.,positive
39996,مش هقولك غير ان نص الضحك اللي ضحكته فى حياتي ك...,positive
39997,ربنا يوفقك ويسهلك وان شاء الله تعدي الفترة دي ...,positive
39998,مبسوطة اوى عملت طريقة مكرونة جديدة و هى دلوقتى...,positive


In [44]:
egypt_df.rename(columns={"review": "sentence"}, inplace=True)

In [45]:
egypt_df['label'].loc[(egypt_df['label'] == "negative")] = 0

In [46]:
egypt_df['label'].loc[(egypt_df['label'] == "positive")] = 2

In [47]:
egypt_df

,sentence,label
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,0
1,دائما اكره اخر ليله في كل مكان .,0
2,يارب اللى يسرق تويتاتى يدخل النار .,0
3,الاسراف فى تناول القهوة يسبب الوفاه .,0
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,2
...,...,...
39995,لنسعد ايامنا بالابتسامة بدلاً ان نملاها بالدموع.,2
39996,مش هقولك غير ان نص الضحك اللي ضحكته فى حياتي ك...,2
39997,ربنا يوفقك ويسهلك وان شاء الله تعدي الفترة دي ...,2
39998,مبسوطة اوى عملت طريقة مكرونة جديدة و هى دلوقتى...,2


In [48]:
egypt_df["label"].value_counts()

0                   19998
2                   19995
positive ‎              2
positive ‎              1
positive​               1
positive​‏              1
negative​‏​‏            1
negative​‏​‏​‏​         1
Name: label, dtype: int64

In [49]:
egypt_df["label"].unique()

array([0, 2, 'positive \u200e    ', 'positive\u200b', 'positive \u200e',
       'positive\u200b\u200f', 'negative\u200b\u200f\u200b\u200f',
       'negative\u200b\u200f\u200b\u200f\u200b\u200f\u200b '],
      dtype=object)

In [50]:
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive \u200e    '], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive\u200b'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive \u200e'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'positive\u200b\u200f'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'negative\u200b\u200f\u200b\u200f'], inplace=True)
egypt_df.drop(egypt_df.index[egypt_df['label'] == 'negative\u200b\u200f\u200b\u200f\u200b\u200f\u200b '], inplace=True)

In [51]:
egypt_df

,sentence,label
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...,0
1,دائما اكره اخر ليله في كل مكان .,0
2,يارب اللى يسرق تويتاتى يدخل النار .,0
3,الاسراف فى تناول القهوة يسبب الوفاه .,0
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .,2
...,...,...
39995,لنسعد ايامنا بالابتسامة بدلاً ان نملاها بالدموع.,2
39996,مش هقولك غير ان نص الضحك اللي ضحكته فى حياتي ك...,2
39997,ربنا يوفقك ويسهلك وان شاء الله تعدي الفترة دي ...,2
39998,مبسوطة اوى عملت طريقة مكرونة جديدة و هى دلوقتى...,2


In [52]:
egypt_df["label"].value_counts()

0    19998
2    19995
Name: label, dtype: int64

#### 1.6 SemEval2017-task4 subtask-A arabic dataset

In [53]:
semeval_df = pd.read_csv("data/sentiment-analysis/SemEval2017-task4-train.subtask-A.arabic.txt", sep="\t", header=None, names=["id", "label", "sentence"])

In [54]:
semeval_df

,id,label,sentence
0,783555835494592513,positive,إجبار أبل على التعاون على فك شفرة اجهزتها http...
1,783582397166125056,positive,RT @20fourMedia: #غوغل تتحدى أبل وأمازون بأجهز...
2,783592390728769536,positive,جوجل تنافس أبل وسامسونج بهاتف جديد https://t.c...
3,783597390070685696,positive,رئيس شركة أبل: الواقع المعزز سيصبح أهم من الطع...
4,783617442031472640,neutral,ساعة أبل في الأسواق مرة أخرى https://t.co/dY2x...
...,...,...,...
3348,785923681432117248,positive,طريقة تعطيل اي زر في لوحة المفاتيح على نظام وي...
3349,785924823860076544,neutral,RT @syst__em: إطلاق تحديث تراكمي برقم 14393.32...
3350,785929366756655104,neutral,4 طرق لحذف الملفات المستعصية من الحذف في ويندو...
3351,785951682467246080,neutral,شرح فيديو مهم جدا حل لمشاكل الكومبيوتر كيف تقو...


In [55]:
semeval_df.drop(["id"], axis=1, inplace=True)

In [56]:
semeval_df = semeval_df.reindex(['sentence','label'], axis=1)

In [57]:
semeval_df

,sentence,label
0,إجبار أبل على التعاون على فك شفرة اجهزتها http...,positive
1,RT @20fourMedia: #غوغل تتحدى أبل وأمازون بأجهز...,positive
2,جوجل تنافس أبل وسامسونج بهاتف جديد https://t.c...,positive
3,رئيس شركة أبل: الواقع المعزز سيصبح أهم من الطع...,positive
4,ساعة أبل في الأسواق مرة أخرى https://t.co/dY2x...,neutral
...,...,...
3348,طريقة تعطيل اي زر في لوحة المفاتيح على نظام وي...,positive
3349,RT @syst__em: إطلاق تحديث تراكمي برقم 14393.32...,neutral
3350,4 طرق لحذف الملفات المستعصية من الحذف في ويندو...,neutral
3351,شرح فيديو مهم جدا حل لمشاكل الكومبيوتر كيف تقو...,neutral


In [58]:
semeval_df['label'].loc[(semeval_df['label'] == "negative")] = 0

In [59]:
semeval_df['label'].loc[(semeval_df['label'] == "neutral")] = 1

In [60]:
semeval_df['label'].loc[(semeval_df['label'] == "positive")] = 2

In [61]:
semeval_df

,sentence,label
0,إجبار أبل على التعاون على فك شفرة اجهزتها http...,2
1,RT @20fourMedia: #غوغل تتحدى أبل وأمازون بأجهز...,2
2,جوجل تنافس أبل وسامسونج بهاتف جديد https://t.c...,2
3,رئيس شركة أبل: الواقع المعزز سيصبح أهم من الطع...,2
4,ساعة أبل في الأسواق مرة أخرى https://t.co/dY2x...,1
...,...,...
3348,طريقة تعطيل اي زر في لوحة المفاتيح على نظام وي...,2
3349,RT @syst__em: إطلاق تحديث تراكمي برقم 14393.32...,1
3350,4 طرق لحذف الملفات المستعصية من الحذف في ويندو...,1
3351,شرح فيديو مهم جدا حل لمشاكل الكومبيوتر كيف تقو...,1


In [62]:
semeval_df["label"].value_counts()

1    1470
0    1141
2     742
Name: label, dtype: int64

#### 1.7 ASTD dataset

Assuming "Objective" label as "Neutral" label.

In [63]:
astd_df = pd.read_csv("data/sentiment-analysis/ASTD.txt", sep="\t", header=None, names=["sentence", "label"])

In [64]:
astd_df

,sentence,label
0,بعد استقالة رئيس #المحكمة_الدستورية ننتظر استق...,OBJ
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,POS
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,NEG
3,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,OBJ
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,NEUTRAL
...,...,...
9689,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,NEG
9690,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,NEG
9691,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",OBJ
9692,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),POS


In [65]:
astd_df["label"].value_counts()

OBJ        6470
NEG        1642
NEUTRAL     805
POS         777
Name: label, dtype: int64

In [66]:
astd_df['label'].loc[(astd_df['label'] == "NEG")] = 0

In [67]:
astd_df['label'].loc[(astd_df['label'] == "NEUTRAL")] = 1

In [68]:
astd_df['label'].loc[(astd_df['label'] == "OBJ")] = 1

In [69]:
astd_df['label'].loc[(astd_df['label'] == "POS")] = 2

In [70]:
astd_df

,sentence,label
0,بعد استقالة رئيس #المحكمة_الدستورية ننتظر استق...,1
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,2
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,0
3,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,1
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,1
...,...,...
9689,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,0
9690,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,0
9691,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",1
9692,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),2


In [71]:
astd_df["label"].value_counts()

1    7275
0    1642
2     777
Name: label, dtype: int64

#### 1.8 Combining the datasets

In [72]:
df_list = [
    tead_pos_df,
    tead_neg_df,
    labr_df,
    arsas_df,
    ajgt_df,
    egypt_df,
    semeval_df,
    astd_df
]

data_df = pd.concat(df_list, axis=0, ignore_index=True)

In [73]:
data_df

,sentence,label
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...,2
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...,2
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم,2
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...,2
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...,2
...,...,...
693912,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,0
693913,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,0
693914,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",1
693915,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),2


In [74]:
data_df["label"].value_counts()

2    462133
0    202725
1     29059
Name: label, dtype: int64

In [75]:
# Saving combined data
data_df.to_csv("data/sentiment-analysis/combined_arabic_data.csv", index=False)

In [76]:
# Testing loading combined data
data_df = pd.read_csv("data/sentiment-analysis/combined_arabic_data.csv")

In [77]:
data_df

,sentence,label
0,مراا حلو ان الاستاذه قايله برسل لكم التخطيط وت...,2
1,نلقى أي قروب يحكي علي ال ندير انسحاب تكتيكي لا...,2
2,يارب شعور إني أكون مروقه ومبسوطه يكون كل يوم,2
3,حاجات تزيد الانثي انوثه تتروش مع رجالها ولا تس...,2
4,وضع الانترنت واجد يضايق لا برودباند البيت ولا ...,2
...,...,...
693912,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,0
693913,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,0
693914,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",1
693915,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),2
